In [14]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dropout
from pandas import DataFrame
import ipaddress

In [15]:
# Define file path and chunk size
file_path = "/mnt/c/Users/mikig/Desktop/UPC/PAE/Datasets/9810e03bba4983da_MOHANAD_A4706/9810e03bba4983da_MOHANAD_A4706/data/NF-UQ-NIDS-v2.csv"
chunk_size = 10000  # Adjust based on your system's memory capacity
nrows = 50000

In [16]:
ds = pd.read_csv(file_path, nrows=nrows)

ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 46 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   IPV4_SRC_ADDR                50000 non-null  object 
 1   L4_SRC_PORT                  50000 non-null  int64  
 2   IPV4_DST_ADDR                50000 non-null  object 
 3   L4_DST_PORT                  50000 non-null  int64  
 4   PROTOCOL                     50000 non-null  int64  
 5   L7_PROTO                     50000 non-null  float64
 6   IN_BYTES                     50000 non-null  int64  
 7   IN_PKTS                      50000 non-null  int64  
 8   OUT_BYTES                    50000 non-null  int64  
 9   OUT_PKTS                     50000 non-null  int64  
 10  TCP_FLAGS                    50000 non-null  int64  
 11  CLIENT_TCP_FLAGS             50000 non-null  int64  
 12  SERVER_TCP_FLAGS             50000 non-null  int64  
 13  FLOW_DURATION_MI

In [17]:
def ip_to_long(ip):
    """
    Convert an IP address to its concatenated zero-padded octet integer representation.

    Args:
        ip (str): IP address in string format.

    Returns:
        int: Integer representation of the concatenated zero-padded octets.
    """
    try:
        return int("".join([octet.zfill(3) for octet in ip.split(".")]))
        # Split the IP address into octets
        octets = ip.split(".")
        # Zero-pad each octet to ensure three digits
        padded_octets = [octet.zfill(3) for octet in octets]
        # Concatenate the padded octets into a single string
        concatenated = "".join(padded_octets)
        # Convert the concatenated string to an integer
        return int(concatenated)
    except ValueError:
        return 0  # Handle invalid IPs by returning 0

In [18]:
ip_columns = ["IPV4_SRC_ADDR", "IPV4_DST_ADDR"]
for col in ip_columns:
    ds[col] = ds[col].apply(ip_to_long)

ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 46 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   IPV4_SRC_ADDR                50000 non-null  int64  
 1   L4_SRC_PORT                  50000 non-null  int64  
 2   IPV4_DST_ADDR                50000 non-null  int64  
 3   L4_DST_PORT                  50000 non-null  int64  
 4   PROTOCOL                     50000 non-null  int64  
 5   L7_PROTO                     50000 non-null  float64
 6   IN_BYTES                     50000 non-null  int64  
 7   IN_PKTS                      50000 non-null  int64  
 8   OUT_BYTES                    50000 non-null  int64  
 9   OUT_PKTS                     50000 non-null  int64  
 10  TCP_FLAGS                    50000 non-null  int64  
 11  CLIENT_TCP_FLAGS             50000 non-null  int64  
 12  SERVER_TCP_FLAGS             50000 non-null  int64  
 13  FLOW_DURATION_MI

In [19]:
# Initialize lists to store processed data
X_list = []
y_list = []

ip_columns = ["IPV4_SRC_ADDR", "IPV4_DST_ADDR"]

# Process data in chunks
for chunk_id, chunk in enumerate(
    pd.read_csv(file_path, chunksize=chunk_size, nrows=nrows)
):
    # Drop unnecessary columns
    chunk = chunk.drop(columns=["Label", "Dataset"])

    # Process the ip columns
    for col in ip_columns:
        chunk[col] = chunk[col].apply(ip_to_long)

    # Separate features and target
    X_chunk = chunk.drop(columns=["Attack"])
    y_chunk = chunk["Attack"]

    # Append to lists
    X_list.append(X_chunk)
    y_list.append(y_chunk)

# Concatenate all chunks
X = pd.concat(X_list)
y = pd.concat(y_list)

In [20]:
# One-hot encode the target variable
y = pd.get_dummies(y).values

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [21]:
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation="relu"))
model.add(Dense(32, activation="relu"))
model.add(Dense(y_train.shape[1], activation="softmax"))

/home/miquel/.local/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [22]:
# Compile the model
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10


I0000 00:00:1735382220.235223    5136 service.cc:148] XLA service 0x7f0e74004d00 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1735382220.237911    5136 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2024-12-28 11:37:00.291086: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1735382220.469570    5136 cuda_dnn.cc:529] Loaded cuDNN version 90300


  30/1250 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.2283 - loss: 2.7769    

I0000 00:00:1735382222.434074    5136 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1250/1250 ━━━━━━━━━━━━━━━━━━━━ 13s 8ms/step - accuracy: 0.7928 - loss: 0.8161 - val_accuracy: 0.9045 - val_loss: 15.6591
Epoch 2/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - accuracy: 0.9151 - loss: 0.2777 - val_accuracy: 0.9250 - val_loss: 18.2409
Epoch 3/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - accuracy: 0.9285 - loss: 0.2233 - val_accuracy: 0.9355 - val_loss: 18.2536
Epoch 4/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - accuracy: 0.9404 - loss: 0.1982 - val_accuracy: 0.9434 - val_loss: 18.4948
Epoch 5/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.9491 - loss: 0.1714 - val_accuracy: 0.9490 - val_loss: 22.3357
Epoch 6/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.9505 - loss: 0.1643 - val_accuracy: 0.9477 - val_loss: 16.8066
Epoch 7/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - accuracy: 0.9514 - loss: 0.1561 - val_accuracy: 0.9498 - val_loss: 18.6241
Epoch 8/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - accuracy: 0.9555 - loss: 0.1517 - val

In [23]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.3f}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9519 - loss: 3.5681
Test Accuracy: 0.950
